# Predicción de Churn Bancario con MLOps
---
### Objetivo

Este notebook demuestra la ejecución completa del proyecto `churn-predictor-dormammu`, cuyo objetivo es predecir la probabilidad de que un cliente bancario abandone (churn) usando aprendizaje automático. El flujo de trabajo ha sido estructurado utilizando buenas prácticas de **MLOps**, incluyendo:

* Procesamiento modular del dataset
* Entrenamiento y evaluación de modelos con `scikit-learn`
* Registro de métricas, artefactos y versiones con `MLflow`
* Uso de una CLI para automatización vía `argparse`
* Organización profesional en scripts `.py` reutilizables

### Dataset

Los datos provienen de un conjunto simulado estilo Monopoly, disponible en Kaggle como:

> `nadiaarellanog/base-clientes-monopoly`

Incluyen información demográfica, comportamiento financiero y transaccional mensual por cliente.

### ¿Qué se demuestra aquí?

Este notebook está diseñado como una **demo interactiva** del proyecto, y recorre paso a paso los siguientes bloques:

1. **Carga de datos** desde Kaggle usando `kagglehub`
2. **Preprocesamiento**, limpieza y transformación de variables
3. **Generación de variable target**
4. **Entrenamiento de un modelo `RandomForestClassifier`**
5. **Evaluación del modelo con métricas y visualizaciones**
6. **Registro de resultados con `MLflow` Tracking Server**

> **Nota**: Este notebook no duplica código. Importa y ejecuta directamente los módulos Python del pipeline.

## Configuración Inicial

Para garantizar la reproducibilidad y evitar conflictos entre paquetes, se recomienda crear un entorno virtual antes de ejecutar este notebook. 

### 1. Crear entorno virtual (opcional pero recomendado)

```bash
python -m venv venv
source venv/bin/activate  # Linux/macOS
venv\Scripts\activate     # Windows
````

### 2. Instalar dependencias

```bash
pip install -r requirements.txt
```

---

Una vez completado este paso, puedes ejecutar el notebook sin problemas. El entorno virtual también permite trabajar con MLflow y otros componentes sin interferencias del sistema.


In [ ]:
# Permite importar src desde notebooks/
import sys
from pathlib import Path

# Agrega la carpeta raíz del proyecto al sys.path
root_path = Path().resolve().parent
if str(root_path) not in sys.path:
    sys.path.append(str(root_path))

In [ ]:
# Cargar módulos necesarios
import os
import mlflow
import pandas as pd
import matplotlib.pyplot as plt

# Cargar módulos del proyecto
from src.load_data import descargar_dataset, cargar_datos
from src.preprocessing import limpiar_columnas, eliminar_outliers, escalar_variables
from src.feature_engineering import calcular_target
from src.model_training import dividir_datos, entrenar_modelo, guardar_modelo
from src.evaluation import evaluar_modelo

# Visualización de gráficos dentro del notebook
%matplotlib inline

## Carga de Datos

In [ ]:
# Importar funciones desde el pipeline
from src.load_data import descargar_dataset, cargar_datos

# Descargar y cargar los datos
print("[INFO] Descargando y cargando dataset...")
path = descargar_dataset()
df = cargar_datos(path)

# Vista previa
df.head()

## Limpieza y transformación inicial

Este bloque ejecuta las funciones encargadas de preparar el dataset antes del feature engineering. Estas funciones están definidas en ```src/preprocessing.py``` y encapsulan tareas como:

* Limpieza de columnas vacías o irrelevantes
* Corrección de tipos
* Eliminación de outliers
* Estandarización/normalización de columnas

In [ ]:
from src.preprocessing import limpiar_columnas, eliminar_outliers, escalar_variables

# Limpiar columnas irrelevantes, renombrar o corregir tipos
print("[INFO] Limpiando columnas...")
df = limpiar_columnas(df)

# Eliminar outliers en columnas críticas
outlier_cols = ['Renta', 'CUPO_L1', 'CUPO_L2', 'CUPO_MX']
df = eliminar_outliers(df, outlier_cols)

# Estandarizar variables
tx_cols = [f'Txs_T{i:02}' for i in range(1, 13) if f'Txs_T{i:02}' in df.columns]
std_cols = ['Renta', 'CUPO_L1', 'CUPO_L2', 'CUPO_MX']
df = escalar_variables(df, std_cols, tx_cols)

# Verificar estructura del DataFrame
df.info()

## Feature Engineering

Aquí generamos variables nuevas a partir de las ya existentes, incluyendo la más importante: la variable objetivo (target). En este proyecto, esa variable es total_transacciones_anual_log y su versión binaria target_binario, que son derivadas de las transacciones históricas.

---

El cálculo aplica una transformación logarítmica para suavizar la distribución sesgada, y también clasifica los valores según percentiles para facilitar el modelado como problema de clasificación binaria.

In [ ]:
from src.feature_engineering import calcular_target

# Calcular variable objetivo (log-transform y clasificación)
print("[INFO] Calculando variable target...")
df = calcular_target(df)

# Revisión rápida
df[["total_transacciones_anual", "total_transacciones_anual_log", "target_binario"]].describe()

## Cargar Modelo Previamente Entrenado (opcional)

Si ya has entrenado el modelo y lo guardaste anteriormente, puedes cargarlo aquí en lugar de volver a entrenar.


In [ ]:
import os
import joblib

modelo_path = "../models/modelo_rf.pkl"  # Ruta desde el notebook

if os.path.exists(modelo_path):
    modelo_dict = joblib.load(modelo_path)
    modelo = modelo_dict["modelo"]
    input_example = modelo_dict["input_example"]
    signature = modelo_dict["signature"]
    print("[SUCCESS] Modelo cargado exitosamente desde 'models/modelo_rf.pkl'")
    entrenar = False
else:
    print("/!\ No se encontró modelo previamente guardado. Procede al entrenamiento.")
    entrenar = True

## División de Datos y Entrenamiento del Modelo

Aquí dividimos el conjunto de datos en entrenamiento y prueba, y luego entrenamos un modelo ```RandomForestClassifier```. El procesamiento de variables incluye escalado de variables numéricas y codificación one-hot para variables categóricas.

In [ ]:
from src.model_training import dividir_datos, entrenar_modelo

# Dividir datos
X_train, X_test, y_train, y_test = dividir_datos(df)

# Identificar columnas numéricas y categóricas
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.difference([
    'total_transacciones_anual', 'total_transacciones_anual_log', 'target_binario'
]).tolist()

# Eliminar columnas categóricas con demasiados niveles
cat_cols = [col for col in cat_cols if df[col].nunique() <= 100]

# Entrenar modelo
modelo, input_example, signature = entrenar_modelo(
    X_train, y_train,
    cat_cols, num_cols,
    n_estimators=100,
    max_depth=None,
    verbose=1
)

## Evaluación del Modelo y Seguimiento con MLflow

En esta etapa, evaluamos el rendimiento del modelo utilizando métricas comunes (precision, recall, F1-score) y registramos todo en MLflow: modelo, parámetros, métricas y artefactos (como la matriz de confusión).

### Configuración del servidor MLflow

Para asegurar el registro correcto de experimentos, este proyecto usa un servidor local de MLflow. Antes de ejecutar el pipeline, debes iniciar MLflow con el siguiente comando (desde la raíz del proyecto):

#### Windows (CMD o PowerShell):

```bash
mlflow server ^
  --backend-store-uri sqlite:///mlflow.db ^
  --default-artifact-root file:///C:/Users/TU_USUARIO/churn-predictor-dormammu/mlruns ^
  --host 127.0.0.1 ^
  --port 5000
````

> Reemplaza `TU_USUARIO` con tu nombre de usuario real en Windows si copias el proyecto.

Una vez lanzado, abre [http://127.0.0.1:5000](http://127.0.0.1:5000) para acceder a la interfaz de MLflow.

---


In [ ]:
import mlflow
mlflow.end_run()

mlflow.set_tracking_uri("http://127.0.0.1:5000")  # debe coincidir con el puerto del servidor
mlflow.set_experiment("Churn-Predictor-Dormammu")

In [ ]:
from src.evaluation import evaluar_modelo
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn

# Iniciar run en MLflow
with mlflow.start_run(run_name="RandomForest_Churn_Clasificacion"):
    mlflow.set_tag("author", "alex-msu")  # Cambia si gustas

    # Loguear parámetros
    mlflow.log_param("modelo", "RandomForestClassifier")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", None)

    # Loguear el modelo
    mlflow.sklearn.log_model(
        sk_model=modelo,
        name="modelo_rf",
        input_example=input_example,
        signature=signature
    )

    # Evaluar
    resultados, y_pred = evaluar_modelo(modelo, X_test, y_test)

    # Loguear métricas
    for k, v in resultados.items():
        mlflow.log_metric(k, v)

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues')
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")
    plt.close()

    # Clasification Report
    report = classification_report(y_test, y_pred, output_dict=True)
    mlflow.log_metric("precision", report["weighted avg"]["precision"])
    mlflow.log_metric("recall", report["weighted avg"]["recall"])
    mlflow.log_metric("f1_score", report["weighted avg"]["f1-score"])


## Guardado del Modelo (opcional)

Aunque el modelo ya está logueado en MLflow, puede ser útil guardar una copia local como respaldo o para pruebas offline. Esta sección muestra cómo hacerlo con joblib.

In [ ]:
import joblib
import os

# Ruta al directorio raíz (una carpeta arriba)
root_model_path = os.path.abspath(os.path.join("..", "models"))
os.makedirs(root_model_path, exist_ok=True)

# Guardar modelo
joblib.dump({
    "modelo": modelo,
    "input_example": input_example,
    "signature": signature
}, os.path.join(root_model_path, "modelo_rf.pkl"))

print("[SUCCESS] Modelo guardado correctamente en '../models/modelo_rf.pkl'")

## Conclusión

Este flujo demuestra un pipeline completo de clasificación de churn con MLOps:
- Preprocesamiento estructurado
- Entrenamiento reproducible
- Evaluación clara
- Registro completo con MLflow

Puedes ahora:
- Comparar runs en la UI de MLflow (`http://localhost:5000`)
- Versionar y cargar modelos entrenados
- Extender el proyecto con validación cruzada, grid search, alertas o despliegue en API

---